# Project 1 - Iowa Liquor 

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and wants a report of current liquor sales by county and projections for the rest of the year. 

Your task is as follows:

* Calculate the yearly liquor sales for each store using the provided data. You can add up the transactions for each year, and store sales in 2015 specifically will be used later as your target variable.
* Use the data from 2015 to make a linear model using as many variables as you find useful to predict the yearly sales of all stores. You must use the sales from Jan to March as one of your variables.
* Use your model for 2015 to estimate total sales in 2016, extrapolating from the sales so far for Jan-March of 2016.
* Report your findings, including any projected increase or decrease in total sales (over the entire state) for the tax committee of the Iowa legislature.
* Use cross-validation to check how your model predicts to held out data compared to the model metrics on the full dataset.
* Fit your model(s) using one or both of the regularization tactics covered. Explain whether the regularized or the non-regularized model performed better and what the selected regression(s) are doing.



# Part 2

### Feature Engineering, Model Building, and Tuning

In Part 2 of this two-part project, you will use the insights gained from your Exploratory Data Analysis (EDA) to build a linear regression model predicting end-of-year total sales using Q1 data. You will use 2015 data to train and tune your model, then make final predictions using Q1 2016 data to make your best estimates for end of year 2016!

### Requirements:


**Mine the data**
- Create necessary derived columns from the data
- Format, clean, slice, and combine the data in Python

**Build a data model**
- Complete linear regressions using scikit-learn or statsmodels and interpret your findings
- Calculate and plot predicted probabilities and/or present tables of results
- Describe the bias-variance tradeoff of your model and errors metrics
- Evaluate model fit by using loss functions, including mean absolute error, mean squared error, and root mean squared error, or r-squared

**Present the results**
- Create a Jupyter Notebook hosted on GitHub that provides a dataset overview with visualizations, statistical analysis, data cleaning methodologies, and models
- Create a write-up on the interpretation of findings including an executive summary with conclusions and next steps

***Bonus!:***
- Handle outliers, use regularization (Ridge & Lasso regressions)
- Brainstorm ways to improve your analysis; for example:
 - Add additional breakdowns and models, e.g. by month.
 - Recommend additional data that might improve your models
 - Can you think of other uses for the dataset? E.g healthcare / disease estimates

In [ ]:
# Made it to the kfolds task and will continue to work on the assignment until complete

In [196]:
import os

os.path.isfile('../Assets/Iowa_Liquor_sample.csv') 

True

In [197]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Import and Clean data:

This time, we've cleaned the data set and column names for you; however we have not touched missing values.

In [198]:
# Import, convert 'Date' col to datetime
lq = pd.read_csv('../Assets/Iowa_Liquor_sample.csv',parse_dates=['Date'],infer_datetime_format=True)

In [199]:
# format column names
import re

liquor.columns = [re.sub("[^a-zA-Z]+", "", x) for x in liquor.columns]

In [50]:
lq.columns.values

array(['Date', 'Store Number', 'City', 'Zip Code', 'County Number',
       'County', 'Category', 'Category Name', 'Vendor Number',
       'Item Number', 'Item Description', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'], dtype=object)

In [200]:
# remove spaces
lq.columns = lq.columns.str.replace(' ','')

In [201]:
# remove both left and right parentheses
lq.columns = lq.columns.str.replace('(','')
lq.columns = lq.columns.str.replace(')','')
lq.columns.values

array(['Date', 'StoreNumber', 'City', 'ZipCode', 'CountyNumber', 'County',
       'Category', 'CategoryName', 'VendorNumber', 'ItemNumber',
       'ItemDescription', 'BottleVolumeml', 'StateBottleCost',
       'StateBottleRetail', 'BottlesSold', 'SaleDollars',
       'VolumeSoldLiters', 'VolumeSoldGallons'], dtype=object)

In [201]:
lq.dtypes

Date                 datetime64[ns]
StoreNumber                   int64
City                         object
ZipCode                      object
CountyNumber                float64
County                       object
Category                    float64
CategoryName                 object
VendorNumber                  int64
ItemNumber                    int64
ItemDescription              object
BottleVolumeml                int64
StateBottleCost              object
StateBottleRetail            object
BottlesSold                   int64
SaleDollars                  object
VolumeSoldLiters            float64
VolumeSoldGallons           float64
dtype: object

In [141]:
lq.head(1)

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38


In [202]:
# remove '$' in values and convert to numeric
adjust_cols = ['StateBottleCost','StateBottleRetail','SaleDollars']

for col in adjust_cols:
    lq[col] = pd.to_numeric(lq[col].str.replace('$',''),errors='coerce')

In [340]:
lq.head(1)

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.5,6.75,12,81.0,9.0,2.38


### Null Values

Handle null values as you see fit

In [203]:
# liquor = liquor.dropna()
# liquor.isnull().sum()

# remove spaces
lq = lq.dropna(how = 'any')

In [58]:
lq.isnull().sum()

Date                 0
StoreNumber          0
City                 0
ZipCode              0
CountyNumber         0
County               0
Category             0
CategoryName         0
VendorNumber         0
ItemNumber           0
ItemDescription      0
BottleVolumeml       0
StateBottleCost      0
StateBottleRetail    0
BottlesSold          0
SaleDollars          0
VolumeSoldLiters     0
VolumeSoldGallons    0
dtype: int64

In [204]:
lq[('Category')] = lq[('Category')].astype(int)

In [60]:
lq.dtypes

Date                 datetime64[ns]
StoreNumber                   int64
City                         object
ZipCode                      object
CountyNumber                float64
County                       object
Category                      int32
CategoryName                 object
VendorNumber                  int64
ItemNumber                    int64
ItemDescription              object
BottleVolumeml                int64
StateBottleCost             float64
StateBottleRetail           float64
BottlesSold                   int64
SaleDollars                 float64
VolumeSoldLiters            float64
VolumeSoldGallons           float64
dtype: object

## Split Data to Create a features and targets

The goal of this project is to predict **total year-end 2015 sales for each store** using **first-quarter 2015 data**

Our data is currently formatted as total purchases for each product per day per store for every day in the year. We will need to group our data by store when we perform our aggregations.

In order to accomplish our goal, we need two sets of data:
* Total full-year  sales for each store in 2015 (our target / y)
* Data from Q1 2015 (will become our features / X)

Create two dataframes, 'liquor2015_fy' and 'liquor2015_q1'

'liquor2015_fy' should contain only store numbers and the full year sales for that store

'liquor2015_q1' should contain all your features, but only for Q1


In [ ]:
# Calculate the sum of sales for each store in 2015 by grouping the full year data
# hint: what columns do you need? what is your aggregating function? 

In [209]:
# new df lq2015_fy
lq2015_date = lq[(lq['Date'] > '2015-01-03') & (lq['Date'] < '2016-01-01')]

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
208763,2015-01-05,3644,ALTOONA,50009,77.0,Polk,1011200,STRAIGHT BOURBON WHISKIES,259,17956,Evan Williams Str Bourbon,750,7.47,11.21,12,134.52,9.00,2.38
39132,2015-01-05,3882,DAVENPORT,52802,82.0,Scott,1031080,VODKA 80 PROOF,297,35926,Five O'clock PET Vodka,750,3.37,5.06,6,30.36,4.50,1.19
246786,2015-01-05,2569,CEDAR RAPIDS,52402,57.0,Linn,1031200,VODKA FLAVORED,260,77714,Smirnoff Pineapple,750,8.25,12.38,3,37.14,2.25,0.59
223444,2015-01-05,3628,CEDAR RAPIDS,52402,57.0,Linn,1011100,BLENDED WHISKIES,65,24458,Kessler Blend Whiskey,1750,11.02,16.53,6,99.18,10.50,2.77
182000,2015-01-05,2666,ANKENY,50023,77.0,Polk,1022100,TEQUILA,395,89191,Jose Cuervo Especial Reposado Tequila Mini,500,11.50,17.25,2,34.50,1.00,0.26
231234,2015-01-05,3806,WEST DES MOINES,50266,77.0,Polk,1011200,STRAIGHT BOURBON WHISKIES,65,27544,Red Stag By Jim Beam,750,11.03,16.55,12,198.60,9.00,2.38
105047,2015-01-05,4617,DES MOINES,50317,77.0,Polk,1081600,WHISKEY LIQUEUR,260,66208,Piehole Cherry Pie,750,7.49,11.24,4,44.96,3.00,0.79
241228,2015-01-05,2521,WEST DES MOINES,50265,77.0,Polk,1062310,SPICED RUM,260,43337,Captain Morgan Spiced Rum,1000,11.75,17.63,12,211.56,12.00,3.17
223373,2015-01-05,4222,EVANSDALE,50707,7.0,Black Hawk,1081300,PEPPERMINT SCHNAPPS,434,81208,Paramount Peppermint Schnapps,1750,7.08,10.62,6,63.72,10.50,2.77
39139,2015-01-05,2633,DES MOINES,50320,77.0,Polk,1031200,VODKA FLAVORED,260,41076,Jeremiah Weed Sweet Tea Vodka,750,9.65,14.48,36,521.28,27.00,7.13


In [265]:
# lq2015_date.sort_values(by='Date')
lq2015_date.Date.sort_values(ascending=False)

25105    2015-12-31
92124    2015-12-31
169695   2015-12-31
79767    2015-12-31
25176    2015-12-31
110265   2015-12-31
205347   2015-12-31
110138   2015-12-31
167883   2015-12-31
149323   2015-12-31
264182   2015-12-31
99447    2015-12-31
173297   2015-12-31
174554   2015-12-31
133753   2015-12-31
164408   2015-12-31
79423    2015-12-31
121947   2015-12-31
265777   2015-12-31
228716   2015-12-31
49494    2015-12-30
49048    2015-12-30
75550    2015-12-30
21357    2015-12-30
265685   2015-12-30
189654   2015-12-30
108720   2015-12-30
141727   2015-12-30
49102    2015-12-30
80350    2015-12-30
            ...    
122124   2015-01-05
154619   2015-01-05
143490   2015-01-05
168956   2015-01-05
122074   2015-01-05
117328   2015-01-05
39299    2015-01-05
270128   2015-01-05
143125   2015-01-05
23568    2015-01-05
39058    2015-01-05
182171   2015-01-05
170405   2015-01-05
39027    2015-01-05
117246   2015-01-05
76835    2015-01-05
254053   2015-01-05
264211   2015-01-05
29253    2015-01-05


In [215]:
lq2015_fy = lq2015_date.groupby(lq2015_date.StoreNumber).agg({'SaleDollars':sum})

In [335]:
lq2015_fy.head()

,SaleDollars
StoreNumber,
2106,146038.70
2113,9310.22
2130,111583.91
2152,7721.08
2178,24324.18


In [246]:
lq2015_fy.SaleDollars.sort_values(ascending=False)

StoreNumber
2633    997924.42
4829    874664.18
3385    439584.35
2512    419534.61
3420    348837.36
3952    331874.38
3814    247417.42
3354    240153.72
3773    232066.78
2670    227060.96
2593    191192.40
3447    187650.04
2629    184362.74
5102    184330.16
2625    181481.92
3524    180805.62
3820    172792.22
2663    171383.33
4167    167173.55
2648    158428.10
2561    157134.87
2616    154160.15
4677    151432.73
2502    150094.01
2500    149121.63
2106    146038.70
4312    145075.52
2501    144833.66
2619    143958.96
2506    143431.66
          ...    
4954       570.60
4985       531.96
4929       528.24
5152       520.74
5212       504.54
5206       500.90
4737       493.56
5020       472.08
5161       463.40
4961       450.01
4990       420.81
4776       376.56
4121       375.06
5178       336.07
4778       305.88
5191       303.90
5186       298.86
5201       289.78
4834       270.72
5175       224.91
5213       180.00
5056       174.65
5193       139.50
4059       105.4

In [258]:
# new df lq2015_q1
lq2015_q1 = lq[(lq['Date'] > '2015-01-03') & (lq['Date'] < '2015-04-01')]

In [263]:
# lq2015_q1.sort_values(by='Date')
lq2015_q1.Date.sort_values(ascending=False)

137888   2015-03-31
62035    2015-03-31
252962   2015-03-31
155461   2015-03-31
28333    2015-03-31
63255    2015-03-31
197780   2015-03-31
197880   2015-03-31
197886   2015-03-31
197938   2015-03-31
44183    2015-03-31
266774   2015-03-31
198145   2015-03-31
198165   2015-03-31
252776   2015-03-31
103029   2015-03-31
54732    2015-03-31
7138     2015-03-31
197204   2015-03-31
156019   2015-03-31
196238   2015-03-31
156273   2015-03-31
28919    2015-03-31
195781   2015-03-31
195866   2015-03-31
156192   2015-03-31
258840   2015-03-31
196264   2015-03-31
196997   2015-03-31
156080   2015-03-31
            ...    
149614   2015-01-05
149407   2015-01-05
68277    2015-01-05
90590    2015-01-05
11970    2015-01-05
184009   2015-01-05
117682   2015-01-05
149466   2015-01-05
184026   2015-01-05
167789   2015-01-05
12011    2015-01-05
51897    2015-01-05
57879    2015-01-05
184128   2015-01-05
167720   2015-01-05
167381   2015-01-05
167712   2015-01-05
134206   2015-01-05
184283   2015-01-05


# Feature Engineering

Using the insight your gained into your dataset while perfomorming *exploratory data analysis* in Part 1 of the project, aggregate the liquor2015_q1 data frame to create cross-sectional features from our longitudinal data.


[Aggregation functions in pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)

In addition to aggregation, you may chose to create columns to more advanced measures of the data, such as sales for a particular product or category, measures of profitbility, daily or weekly sales statistics, etc.

Combine your aggregations and other engineered features into a dataframe called 'liquor2015_q1_features'

*At a minimum, you will need to aggreate your features by Store in order to procede*



In [ ]:
# I put the columns into Excel and wrote notes on what values I thought would work best for predictions
# Started out by creating a lot of correlation maps and tables. Crazy scores. You pointed out multicolienarity. Aha moment. New process below.

In [ ]:
# Notes: possible new columns
# Relationship between mean Bottles Sold and average SaleDollars (by store)
# There is a strong possitive relationship between the average number of bottles sold per transaction at a store and the average total sale amount per transaction at the same store.
# Groupby store and sum bottles sold and sale dollars per week

In [294]:
# Remove columns that will not be used in the model
# Create new df "drop"
lq2015_q1_drop = lq2015_q1.drop(['City', 'ZipCode', 'CountyNumber', 'County','CategoryName', 'VendorNumber','ItemDescription', 'BottleVolumeml'], axis=1)

In [295]:
lq2015_q1_drop.head()

,Date,StoreNumber,Category,ItemNumber,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
12,2015-01-22,3858,1012100,15248,8.92,13.38,1,13.38,1.75,0.46
19,2015-03-04,2614,1012100,13638,9.97,14.96,6,89.76,10.50,2.77
20,2015-01-16,2590,1031200,41474,14.25,21.38,2,42.76,1.50,0.40
21,2015-01-13,3942,1012100,11786,5.23,7.85,12,94.20,9.00,2.38
22,2015-03-24,3981,1011100,25606,7.00,10.50,12,126.00,9.00,2.38


In [312]:
# Sort functions
# lq2015_q1_features.sort_values(by='Date')
# lq2015_q1.sort_values(by = 'Category', ascending=False)
# lq2015_q1_features.sort_values(by = 'StoreNumber', ascending=False)

,Date,StoreNumber,Category,ItemNumber,StateBottleCost,StateBottleRetail,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
107866,2016-03-08,9023,1011200.0,26996,17.59,26.39,633.36,18.0,4.76
139261,2015-10-27,9018,1062310.0,46686,14.30,21.45,6435.00,225.0,59.44
105091,2015-06-04,9013,1031080.0,36447,12.54,18.81,677.16,27.0,7.13
89110,2016-01-20,9013,1031080.0,38058,7.18,10.77,129.24,12.0,3.17
28118,2015-11-19,9013,1011100.0,27255,3.09,4.64,556.80,90.0,23.78
100716,2015-12-01,9013,1011100.0,27629,9.09,13.64,818.40,45.0,11.89
236012,2016-03-09,9013,1101100.0,77284,6.45,9.68,232.32,18.0,4.76
42383,2015-08-04,9013,1011100.0,27255,9.17,13.76,165.12,9.0,2.38
16295,2015-01-20,9010,1081600.0,75740,13.29,19.94,478.56,18.0,4.76
141678,2015-05-20,9010,1081600.0,75740,13.29,19.94,239.28,9.0,2.38


In [296]:
# Add calculated measure
lq2015_q1_drop['Sold_div_Sales'] = ((lq2015_q1_drop.SaleDollars / lq2015_q1_drop.BottlesSold))

In [309]:
lq2015_q1_drop.head()

,Date,StoreNumber,Category,ItemNumber,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons,Sold_div_Sales
12,2015-01-22,3858,1012100,15248,8.92,13.38,1,13.38,1.75,0.46,13.38
19,2015-03-04,2614,1012100,13638,9.97,14.96,6,89.76,10.50,2.77,14.96
20,2015-01-16,2590,1031200,41474,14.25,21.38,2,42.76,1.50,0.40,21.38
21,2015-01-13,3942,1012100,11786,5.23,7.85,12,94.20,9.00,2.38,7.85
22,2015-03-24,3981,1011100,25606,7.00,10.50,12,126.00,9.00,2.38,10.50


In [302]:
# Create new df "f2"
lq2015_q1_features = lq2015_q1_drop.groupby(lq2015_date.StoreNumber).agg({'SaleDollars':'mean', 'BottlesSold':'mean', 'VolumeSoldLiters':'mean', 'VolumeSoldGallons':'mean', 'Sold_div_Sales':'mean'}) [['BottlesSold','VolumeSoldGallons','VolumeSoldLiters','SaleDollars','Sold_div_Sales']]

In [320]:
lq2015_q1_features.head()

,BottlesSold,VolumeSoldGallons,VolumeSoldLiters,SaleDollars,Sold_div_Sales
StoreNumber,,,,,
2106,20.968992,5.173721,19.582171,304.552636,15.075271
2113,4.666667,1.114286,4.216905,67.458333,15.821190
2130,17.620690,4.394598,16.635057,278.995057,15.401379
2152,4.812500,1.252500,4.741875,62.608125,14.460938
2178,10.208333,2.255833,8.537708,122.008542,14.748542


In [304]:
lq2015_q1_features.describe()

,BottlesSold,VolumeSoldGallons,VolumeSoldLiters,SaleDollars,Sold_div_Sales
count,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000
mean,10.234351,2.288141,8.660153,118.714201,13.536197
std,7.019197,1.932096,7.313667,112.491715,3.647096
min,1.000000,0.130000,0.500000,11.210000,1.700000
25%,6.388889,1.401250,5.306429,70.750000,11.395862
50%,9.047872,1.978000,7.483333,100.023571,13.553143
75%,12.000000,2.603333,9.857143,140.347500,15.240465
max,104.608696,31.942609,120.913043,2454.070435,47.774444


In [305]:
lq2015_q1_features.shape

(1261, 5)

In [336]:
lq2015_fy.head()

,SaleDollars
StoreNumber,
2106,146038.70
2113,9310.22
2130,111583.91
2152,7721.08
2178,24324.18


## Combine Q1 Features with Full Year Target

Now that you've created a set of features using the Q1 data, we much combine it wil the full-year data so that our Xs (features) are matched up to their coresponding y's (targets).

Pandas' 'merge' function allows us to combine two dataframes, using SQL-like joins.

[Pandas Merge/Join Documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging)

We will create a new dataframe, called 'liquor2015_combined' by merging our 'liquor2015_fy' and 'liquor2015_q1_features' dataframes on Store Number - giving us a dataframe which in each row has the Q1 features you've developed for each store, and the year-end total sales for that store.

#### In pandas, merge can take two forms:

pd.merge(left_dataframe,right_dataframe, \*\*args)

*or*

left_dataframe.merge(right_dataframe,\*\*args)

Both of these return the merged dataframe. For arguments, you will need to chose which column(s) from your right and left dataframe you're merging on.

Args:
* left: your left-dataframe
* right: your right-dataframe
* on= : if your dataframes have a common column name that you're merging on, use this arg
* left_on= / right_on= : if your dataframes do not have a common column name, you can specify the names
* left_index= / right_index= : these are boolean (True/False) flags for whether to use the dataframe's index as the merging column.



In [ ]:
# Will need to merge > lq2015_fy / lq2015_q1_features

In [317]:
#lq2015_combined = pd.merge(lq2015_q1_features, lq2015_fy, left_on='StoreNumber', right_on='StoreNumber')

In [326]:
lq2015_q1_features.head()

,BottlesSold,VolumeSoldGallons,VolumeSoldLiters,SaleDollars,Sold_div_Sales
StoreNumber,,,,,
2106,20.968992,5.173721,19.582171,304.552636,15.075271
2113,4.666667,1.114286,4.216905,67.458333,15.821190
2130,17.620690,4.394598,16.635057,278.995057,15.401379
2152,4.812500,1.252500,4.741875,62.608125,14.460938
2178,10.208333,2.255833,8.537708,122.008542,14.748542


In [341]:
lq2015_combined = pd.merge(lq2015_q1_features, lq2015_fy, right_index=True, left_index=True)

In [342]:
lq2015_combined.head(1)

,BottlesSold,VolumeSoldGallons,VolumeSoldLiters,SaleDollars_x,Sold_div_Sales,SaleDollars_y
StoreNumber,,,,,,
2106,20.968992,5.173721,19.582171,304.552636,15.075271,146038.7


In [343]:
lq2015_combined.columns = lq2015_combined.columns.str.replace('SaleDollars_x','SaleDollars_q1')
lq2015_combined.columns = lq2015_combined.columns.str.replace('SaleDollars_y','SaleDollars_fy')

In [344]:
lq2015_combined = lq2015_combined.reset_index()

In [358]:
lq2015_combined.head(5)

,StoreNumber,BottlesSold,VolumeSoldGallons,VolumeSoldLiters,SaleDollars_q1,Sold_div_Sales,SaleDollars_fy
0,2106,20.968992,5.173721,19.582171,304.552636,15.075271,146038.70
1,2113,4.666667,1.114286,4.216905,67.458333,15.821190,9310.22
2,2130,17.620690,4.394598,16.635057,278.995057,15.401379,111583.91
3,2152,4.812500,1.252500,4.741875,62.608125,14.460938,7721.08
4,2178,10.208333,2.255833,8.537708,122.008542,14.748542,24324.18


## Cross Validation

As we build our model, we will use cross-validation techniques to help navigate the bias/variance tradeoff, with a goal of producing the best model which will generalize to new data. 

![crossval](../Assets/validation.png)

### Step 1: Hold Out / Testing Data

In order to evaluate our final model performance, we will seperate out a small amount of data which will will not touch while train and test our model (labeled in red as "Testing Data" in the image above). 

In [372]:
from sklearn.model_selection import train_test_split  # model selection rather than cross validation?

In [377]:
# Removed 'shuffle'. Received the following error: TypeError: Invalid parameters passed: {'shuffle': True} 
# training_data,holdout = train_test_split(lq2015_combined,shuffle=True,test_size=0.10,random_state=123)
# without the shuffle attribute
training_data,holdout = train_test_split(lq2015_combined,test_size=0.10,random_state=123)

### Step 2: Kfolds

With our holdout set removed, we can set up **Kfolds** cross validation

In [384]:
# from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold

In [453]:
# Number of folds you wish to trail
# kf = cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True)
# Produced error: NameError: name 'cross_validation' is not defined

# Number of rows in your dataframe
n = training_data.shape[0]

# define 
kf = KFold(n_splits=5, random_state=123, shuffle=True)
# kf = KFold(n, n_folds=5, randomstate=123)

In [454]:
print(kf)

KFold(n_splits=5, random_state=123, shuffle=True)


# Model Building - Linear Regression

With feature prepared and a cross-validation framework in place, train and tune a linear regressor to predict year-end sales using your q1 data

In [455]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import numpy as np

In [442]:
lq2015_combined.columns.values

array(['StoreNumber', 'BottlesSold', 'VolumeSoldGallons',
       'VolumeSoldLiters', 'SaleDollars_q1', 'Sold_div_Sales',
       'SaleDollars_fy'], dtype=object)

In [456]:
#select your feature column names
feature_cols = ['StoreNumber', 'BottlesSold', 'VolumeSoldGallons', 'SaleDollars_q1', 'Sold_div_Sales']

In [457]:
# define your X (features) and y (target)
# hint - make sure your y is not in your X!
X = lq2015_combined[feature_cols]
y = lq2015_combined.SaleDollars_fy

Instantiate your model

In [458]:
lr = LinearRegression()

Use the kfolds iterator to **train** and **evaluate** your model, using Mean Squared Error (MSE) as your evluation metric

In [459]:
# for train,test in kf:
# Set up your training and testing sets
for train,test in kf:
    x_train = X.iloc[train]
    x_test = X.iloc[test]
    y_train = y.iloc[train]
    y_true = y.iloc[test]

# Create a blank list to store fold scores
MSE_score = []

# Fill-in the kfolds-loop:
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # Fit your model on your training x and training y
    lr.fit(x_train,y_train)
    
    # Make Predictions
    y_preds = x_test
    
    # Score your predictions vs. your true values using mean_squared_error
    fold_score = mean_squared_error()
    
    # Append your score 
    MSE_score.append()

TypeError: 'KFold' object is not iterable

In [ ]:
# View your fold scores, and calculate the mean score across your folds
# np.mean()

### Coefficients and Intercept

View the coefficients of your model - what do the coefficients tell you about the relationships between your features and your target?

In [ ]:
list(zip(feature_cols,lr.coef_))

In [ ]:
lr.intercept_

### Tuning Your Model

So far, you've trained a basic linear model and evaluated it using Mean Squared Error. Use the same process as above to evaluate your model using: Mean Absolute Error (MAE), Root Mean Squared Error (RMSE) and calculate the R2 score of your predictions.

Try some of the parameters available for your linear model, and different sets of features to find a model that you feel will **perform best on new, out of sample data**

In [ ]:
feature_cols_new = []

In [ ]:
X = training_data[feature_cols_new]

In [ ]:
# Use K-Folds cross validation to train your model / kfolds loop (define prior to the loop mae.append etc.)
# Evaluate your model using MAE, MSE, RMSE and R2 

In [ ]:
# run a few different feature sets

In [ ]:
# Compare your MAE, MSE, RMSE and R2 values for your folds; describe anything that stands out.
# How do your metrics respond to different feature sets?

In [ ]:
# Evaluate your coefficients and your intercept / reference the previous codeset for the run

## Test against your hold-out set

Before you build your model, you set aside some of your data for testing. Your model has never trained against these data points or been evaluated agaist these points.

Use **ALL** of your training data to train, then test your model against your holdout set.

In [ ]:
# Pick your best set of feature columns
features = [<FEATURES>]

X_train = training_data[features]
y_train = 

x_holdout = holdout[features]
y_holdout = 

In [ ]:
lr = LinearRegression()

In [ ]:
# Fit your model using all of your training data
lr.fit(X_train, y_train)

In [ ]:
# Create predictions using your holdout set (x_holdout)
holdout_preds = lr.predict(x_holdout)

In [ ]:
# score your model using MAE, MSE, RMSE, and R2
# hint: what is y_test and what is your y_true?

# search within sklearn

MAE_score = 
MSE_score =
RMSE_score =
R2_score =


In [ ]:
# print your scores



In [ ]:
# Create a scatter plot of your predicted values vs. their true values
# Describe anything you observe


In [ ]:
# Calculate your residuals (prediction - actual)


In [ ]:
# Create a histogram of your residuals. Describe anything you observe



# Final Predictions

You've created a model that predicts 2015 year end sales based on Q1 2015 data. 

In the data source, we have included data for Q1 of 2016. Apply your feature engineering process to the 2016 Q1 data, then use your trained 2015 model to predict the 2016 year end values for those stores.

Note: you do not have the 2016 year end values to evaluate against.


### Feature Engineering
Perform the same aggregation and feature creation you used on 2015 data on the 2016 data 

In [ ]:
# liquour[liquor.Date.dt.Year == 2016]

In [ ]:
# create new df for 2016

### Make Predictions

Once you have your 2016 features, use your trained 2015 model on the 2016 Q1 data to get your predictions for 2016

Do not retrain a model on the 2016 data*

In [ ]:
# Make Predictions

In [ ]:
# = lr.predict(...new df)

In [ ]:
# Show your 2016 year-end prediction for each store

In [ ]:
# print the variable for lr.predict and create a visualization (bar)

# Evaluation:

Do your best to answer the following questions:

* What was the best set of features you found for your model?
* Describe the relationships between your features and your target
* How did your model perform in the training phase? Against the holdout set? 
* Did it perform better or worse against the holdout set?

Finally:
* Write a short description of your analysis, describing the process you went through and your confidence in your model's predictive ability
* Include any data, or visualizations you feel would help support your findings

# Bonus - Regularization & Grid Search

As a bonus, experiment with the effect of Lasso (L1) and Ridge (L2) regularization on your linear model. Use GridSearch to tune your additional parameters.

See [gridseach 'scoring' options](http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) for a list of scoring function strings recognized by GridSeach

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.grid_search import GridSearchCV

In [ ]:
# Instantiate models # could run these in a loop similiar to kfolds above as one of the methods
lr_ridge = 
lr_lasso = 

In [ ]:
# Use your post-holdout training data, so you can evaluate on the holdout later
X = 
y = 

Expirement with values of Alpha, scoring functions, and L1/L2 regulatization

In [ ]:
params = {'alpha':[0.2,1.0]}

In [ ]:
gs = GridSearchCV(<model>,params,cv=5,scoring='neg_mean_squared_error')

In [ ]:
# Gridsearch incorporates k-folds validation
# You do not have to create training/testing splits
gs.fit(X,y)

In [ ]:
# View all permutation scores
gs.grid_scores_

In [ ]:
# use the best set of parameters
lr_best = gs.best_estimator_

In [ ]:
# try the best estimator on your holdout set

## Evaluation:

Did regularization improve your model? What was the impact of regularization on your features? Did regularization make any features stand out?